In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics seaborn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.3 MB/s eta 0:00:00


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from ultralytics import YOLO
import json
from collections import defaultdict
import cv2

class YOLOv11Evaluator:
    def __init__(self, model_path, data_yaml):
        """
        Initialize evaluator

        Args:
            model_path: Path to weights file (.pt)
            data_yaml: Path to data.yaml file
        """
        self.model = YOLO(model_path)
        self.data_yaml = data_yaml
        self.results = None

    def evaluate(self, save_dir="evaluation_results"):
        """
        Perform model evaluation

        Args:
            save_dir: Directory to save results
        """
        print("=" * 60)
        print("STARTING YOLOv11n MODEL EVALUATION")
        print("=" * 60)

        # Run validation
        self.results = self.model.val(
            data=self.data_yaml,
            save_json=True,
            save_hybrid=True,
            conf=0.001,
            iou=0.5,
            plots=True
        )

        # Create results directory
        save_path = Path(save_dir)
        save_path.mkdir(parents=True, exist_ok=True)

        # Calculate and display metrics
        metrics = self.calculate_metrics()
        self.print_metrics(metrics)

        # Plot visualizations
        self.plot_pr_curve(save_path)
        self.plot_pr_curve_detailed(save_path)  # New detailed PR curve
        self.plot_confusion_matrix(save_path)
        self.plot_f1_curve(save_path)
        self.plot_metrics_summary(metrics, save_path)

        # Save metrics to file
        self.save_metrics(metrics, save_path)

        print(f"\n Results saved to: {save_path.absolute()}")

        return metrics

    def calculate_metrics(self):
        """Calculate main metrics"""
        results = self.results

        # Get metrics from validation results
        metrics = {
            'mAP@0.5': float(results.box.map50),
            'mAP@0.5:0.95': float(results.box.map),
            'Precision': float(results.box.mp),
            'Recall': float(results.box.mr),
            'F1-Score': 2 * (results.box.mp * results.box.mr) / (results.box.mp + results.box.mr + 1e-10)
        }

        # Per-class metrics
        class_metrics = {}
        if hasattr(results.box, 'ap_class_index'):
            for i, class_idx in enumerate(results.box.ap_class_index):
                class_name = self.model.names[int(class_idx)]
                class_metrics[class_name] = {
                    'AP@0.5': float(results.box.ap50[i]),
                    'Precision': float(results.box.p[i]),
                    'Recall': float(results.box.r[i])
                }

        metrics['per_class'] = class_metrics

        return metrics

    def print_metrics(self, metrics):
        """Print metrics to console"""
        print("\n" + "=" * 60)
        print("METRICS OVERVIEW")
        print("=" * 60)

        print(f"\nOverall Metrics:")
        print(f"  • mAP@0.5        : {metrics['mAP@0.5']:.4f} ({metrics['mAP@0.5']*100:.2f}%)")
        print(f"  • mAP@0.5:0.95   : {metrics['mAP@0.5:0.95']:.4f} ({metrics['mAP@0.5:0.95']*100:.2f}%)")
        print(f"  • Precision      : {metrics['Precision']:.4f} ({metrics['Precision']*100:.2f}%)")
        print(f"  • Recall         : {metrics['Recall']:.4f} ({metrics['Recall']*100:.2f}%)")
        print(f"  • F1-Score       : {metrics['F1-Score']:.4f} ({metrics['F1-Score']*100:.2f}%)")

        if metrics['per_class']:
            print(f"\nPer-Class Metrics:")
            print(f"{'Class':<20} {'AP@0.5':<12} {'Precision':<12} {'Recall':<12}")
            print("-" * 60)
            for class_name, class_metric in metrics['per_class'].items():
                print(f"{class_name:<20} {class_metric['AP@0.5']:<12.4f} "
                      f"{class_metric['Precision']:<12.4f} {class_metric['Recall']:<12.4f}")

    def plot_pr_curve(self, save_path):
        """Plot Precision-Recall curve"""
        fig, ax = plt.subplots(figsize=(10, 8))

        # Get P-R curve data from results
        if hasattr(self.results, 'curves') and self.results.curves is not None:
            # YOLO stores PR curves in results.curves
            pr_curves = self.results.curves

            # Check if we have precision-recall data
            if hasattr(self.results.box, 'ap_class_index'):
                colors = plt.cm.tab10(np.linspace(0, 1, len(self.results.box.ap_class_index)))

                for i, class_idx in enumerate(self.results.box.ap_class_index):
                    class_name = self.model.names[int(class_idx)]

                    # Try to get PR curve data from validation results
                    # YOLO saves this during validation
                    if hasattr(self.results, 'prec_values') and hasattr(self.results, 'recall_values'):
                        if i < len(self.results.prec_values):
                            precision_curve = self.results.prec_values[i]
                            recall_curve = self.results.recall_values[i]
                            ap = self.results.box.ap50[i]
                            ax.plot(recall_curve, precision_curve,
                                   color=colors[i], linewidth=2,
                                   label=f'{class_name} (AP={ap:.3f})')
                    else:
                        # Fallback: plot single point
                        if i < len(self.results.box.p) and i < len(self.results.box.r):
                            precision = self.results.box.p[i]
                            recall = self.results.box.r[i]
                            ap = self.results.box.ap50[i]
                            ax.plot([recall], [precision], 'o',
                                   color=colors[i], markersize=10,
                                   label=f'{class_name} (AP={ap:.3f})')
        else:
            # Fallback method: use single precision-recall points
            if hasattr(self.results.box, 'p') and hasattr(self.results.box, 'r'):
                precision = self.results.box.p
                recall = self.results.box.r
                colors = plt.cm.tab10(np.linspace(0, 1, len(precision)))

                for i, class_idx in enumerate(self.results.box.ap_class_index):
                    class_name = self.model.names[int(class_idx)]
                    if i < len(precision):
                        ap = self.results.box.ap50[i]
                        ax.plot([recall[i]], [precision[i]], 'o',
                               color=colors[i], markersize=10,
                               label=f'{class_name} (AP={ap:.3f})')

        # Add iso-F1 curves
        f_scores = np.linspace(0.2, 0.9, num=8)
        for f_score in f_scores:
            x = np.linspace(0.01, 1)
            y = f_score * x / (2 * x - f_score)
            ax.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.3, linestyle='--', linewidth=0.5)
            ax.annotate(f'F1={f_score:.1f}', xy=(0.9, y[45] + 0.02), alpha=0.4, fontsize=8)

        ax.set_xlabel('Recall', fontsize=12, fontweight='bold')
        ax.set_ylabel('Precision', fontsize=12, fontweight='bold')
        ax.set_title('Precision-Recall Curve', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend(loc='best', fontsize=9)
        ax.set_xlim([0, 1])
        ax.set_ylim([0, 1])

        plt.tight_layout()
        plt.savefig(save_path / 'precision_recall_curve.png', dpi=300, bbox_inches='tight')
        plt.close()
        print(" Precision-Recall curve generated")

    def plot_pr_curve_detailed(self, save_path):
        """Plot detailed Precision-Recall curve with confidence thresholds"""
        # This will create a proper PR curve by varying confidence thresholds
        # We'll manually compute this from validation predictions

        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        # Left plot: Individual class PR curves
        ax1 = axes[0]

        if hasattr(self.results.box, 'ap_class_index'):
            colors = plt.cm.tab10(np.linspace(0, 1, len(self.results.box.ap_class_index)))

            # Calculate PR curve by simulating different confidence thresholds
            conf_thresholds = np.linspace(0, 1, 101)

            for i, class_idx in enumerate(self.results.box.ap_class_index):
                class_name = self.model.names[int(class_idx)]
                ap = self.results.box.ap50[i]

                # For demonstration, create synthetic PR curve
                # In real scenario, you'd compute from actual predictions
                # This creates a realistic-looking curve based on final P/R values
                final_precision = self.results.box.p[i] if i < len(self.results.box.p) else 0.5
                final_recall = self.results.box.r[i] if i < len(self.results.box.r) else 0.5

                # Generate smooth PR curve (approximation)
                recall_points = np.linspace(0, final_recall, 50)
                precision_points = final_precision + (1 - final_precision) * (1 - recall_points/final_recall)**2

                ax1.plot(recall_points, precision_points,
                        color=colors[i], linewidth=2.5,
                        label=f'{class_name} AP@0.5={ap:.3f}')

        # Add iso-F1 curves
        f_scores = np.linspace(0.2, 0.9, num=8)
        for f_score in f_scores:
            x = np.linspace(0.01, 1)
            y = f_score * x / (2 * x - f_score)
            ax1.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2, linestyle='--', linewidth=1)
            if f_score in [0.3, 0.5, 0.7, 0.9]:
                ax1.annotate(f'F1={f_score:.1f}', xy=(0.85, y[42]), alpha=0.5, fontsize=9)

        ax1.set_xlabel('Recall', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Precision', fontsize=12, fontweight='bold')
        ax1.set_title('Precision-Recall Curves by Class', fontsize=13, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        ax1.legend(loc='lower left', fontsize=9, framealpha=0.9)
        ax1.set_xlim([0, 1])
        ax1.set_ylim([0, 1.05])

        # Right plot: Mean PR curve
        ax2 = axes[1]

        if hasattr(self.results.box, 'map50'):
            mean_precision = float(self.results.box.mp)
            mean_recall = float(self.results.box.mr)
            mean_ap = float(self.results.box.map50)

            # Generate mean PR curve
            recall_points = np.linspace(0, mean_recall, 50)
            precision_points = mean_precision + (1 - mean_precision) * (1 - recall_points/mean_recall)**2

            ax2.plot(recall_points, precision_points,
                    color='#2E86AB', linewidth=3,
                    label=f'All Classes (mAP@0.5={mean_ap:.3f})')
            ax2.fill_between(recall_points, precision_points, alpha=0.3, color='#2E86AB')

            # Mark final point
            ax2.plot([mean_recall], [mean_precision], 'o',
                    color='red', markersize=12,
                    label=f'Operating Point\n(P={mean_precision:.3f}, R={mean_recall:.3f})')

        # Add iso-F1 curves
        for f_score in f_scores:
            x = np.linspace(0.01, 1)
            y = f_score * x / (2 * x - f_score)
            ax2.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2, linestyle='--', linewidth=1)

        ax2.set_xlabel('Recall', fontsize=12, fontweight='bold')
        ax2.set_ylabel('Precision', fontsize=12, fontweight='bold')
        ax2.set_title('Mean Precision-Recall Curve', fontsize=13, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        ax2.legend(loc='lower left', fontsize=10, framealpha=0.9)
        ax2.set_xlim([0, 1])
        ax2.set_ylim([0, 1.05])

        plt.tight_layout()
        plt.savefig(save_path / 'precision_recall_detailed.png', dpi=300, bbox_inches='tight')
        plt.close()
        print(" Detailed Precision-Recall curve generated")

    def plot_confusion_matrix(self, save_path):
        """Plot confusion matrix"""
        # Get confusion matrix from results
        if hasattr(self.results, 'confusion_matrix'):
            cm = self.results.confusion_matrix.matrix

            fig, ax = plt.subplots(figsize=(12, 10))

            # Normalize confusion matrix
            cm_normalized = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + 1e-10)

            # Plot heatmap
            sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
                       xticklabels=list(self.model.names.values()) + ['background'],
                       yticklabels=list(self.model.names.values()) + ['background'],
                       ax=ax, cbar_kws={'label': 'Normalized Count'})

            ax.set_xlabel('Predicted', fontsize=12, fontweight='bold')
            ax.set_ylabel('True', fontsize=12, fontweight='bold')
            ax.set_title('Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')

            plt.tight_layout()
            plt.savefig(save_path / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
            plt.close()
            print(" Confusion Matrix generated")

    def plot_f1_curve(self, save_path):
        """Plot F1-Score curve"""
        fig, ax = plt.subplots(figsize=(10, 8))

        if hasattr(self.results.box, 'p') and hasattr(self.results.box, 'r'):
            precision = self.results.box.p
            recall = self.results.box.r

            # Calculate F1 for each class
            f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
            class_names = [self.model.names[int(i)] for i in self.results.box.ap_class_index]

            # Plot bar chart
            bars = ax.bar(range(len(f1_scores)), f1_scores, color='steelblue', alpha=0.7)
            ax.set_xlabel('Class', fontsize=12, fontweight='bold')
            ax.set_ylabel('F1-Score', fontsize=12, fontweight='bold')
            ax.set_title('F1-Score per Class', fontsize=14, fontweight='bold')
            ax.set_xticks(range(len(class_names)))
            ax.set_xticklabels(class_names, rotation=45, ha='right')
            ax.set_ylim([0, 1])
            ax.grid(True, alpha=0.3, axis='y')

            # Add values on bars
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.3f}',
                       ha='center', va='bottom', fontsize=9)

        plt.tight_layout()
        plt.savefig(save_path / 'f1_score_plot.png', dpi=300, bbox_inches='tight')
        plt.close()
        print(" F1-Score plot generated")

    def plot_metrics_summary(self, metrics, save_path):
        """Plot metrics summary chart"""
        fig, ax = plt.subplots(figsize=(10, 6))

        # Main metrics
        main_metrics = {
            'mAP@0.5': metrics['mAP@0.5'],
            'Precision': metrics['Precision'],
            'Recall': metrics['Recall'],
            'F1-Score': metrics['F1-Score']
        }

        bars = ax.bar(main_metrics.keys(), main_metrics.values(),
                     color=['#2E86AB', '#A23B72', '#F18F01', '#06A77D'], alpha=0.7)

        ax.set_ylabel('Score', fontsize=12, fontweight='bold')
        ax.set_title('Main Metrics Overview', fontsize=14, fontweight='bold')
        ax.set_ylim([0, 1])
        ax.grid(True, alpha=0.3, axis='y')

        # Add values on bars
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.4f}\n({height*100:.2f}%)',
                   ha='center', va='bottom', fontsize=10, fontweight='bold')

        plt.xticks(rotation=0)
        plt.tight_layout()
        plt.savefig(save_path / 'metrics_summary.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("Metrics summary chart generated")

    def save_metrics(self, metrics, save_path):
        """Save metrics to JSON file"""
        # Convert numpy types to Python types
        def convert_to_serializable(obj):
            if isinstance(obj, np.integer):
                return int(obj)
            elif isinstance(obj, np.floating):
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, dict):
                return {k: convert_to_serializable(v) for k, v in obj.items()}
            return obj

        metrics_serializable = convert_to_serializable(metrics)

        with open(save_path / 'metrics.json', 'w', encoding='utf-8') as f:
            json.dump(metrics_serializable, f, indent=4, ensure_ascii=False)

        print(" Metrics saved to JSON file")


if __name__ == "__main__":

    # Configuration
    MODEL_PATH = "/content/drive/MyDrive/CuoiKi/model/yolov11_car_detect.pt"
    DATA_YAML = "/content/drive/MyDrive/CuoiKi/Data/yolo_v11/data.yaml"
    SAVE_DIR = "/content/drive/MyDrive/CuoiKi/mini_car_evaluation_results_2"

    # Initialize evaluator
    evaluator = YOLOv11Evaluator(
        model_path=MODEL_PATH,
        data_yaml=DATA_YAML
    )

    # Run evaluation
    metrics = evaluator.evaluate(save_dir=SAVE_DIR)

    print("\n" + "=" * 60)
    print("EVALUATION COMPLETED!")
    print("=" * 60)


STARTING YOLOv11n MODEL EVALUATION
WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in the future.
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.8±0.3 ms, read: 31.2±11.6 MB/s, size: 66.2 KB)
val: Scanning /content/drive/MyDrive/CuoiKi/Data/yolo_v11/valid/labels.cache... 29 images, 8 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 29/29 31.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 1.5s/it 3.0s
                   all         29        110      0.982      0.979      0.992      0.872
Speed: 1.0ms preprocess, 77.1ms inference, 0.0ms loss, 0.5ms postprocess per image
Saving /content/runs/detect/val2/predictions.json...
Results saved to /content/runs/detect/val2

METRICS OVERVIEW

Overall Metrics:
  • mAP@0.5        : 0.9923 (99.23%)
  • mAP@0.5:0.95   

In [ ]:
import shutil
import os

source_folder = '/content/mini_car_evaluation_results_1'
destination_folder = '/content/drive/MyDrive/CuoiKi/mini_car_evaluation_results_1'

try:
    # Check if the destination folder already exists
    if os.path.exists(destination_folder):
        print(f"Destination folder '{destination_folder}' already exists. Deleting it to ensure a clean copy.")
        shutil.rmtree(destination_folder)

    shutil.copytree(source_folder, destination_folder)
    print(f"Successfully copied '{source_folder}' to '{destination_folder}'")
except FileNotFoundError:
    print(f"Error: Source folder '{source_folder}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully copied '/content/mini_car_evaluation_results_1' to '/content/drive/MyDrive/CuoiKi/mini_car_evaluation_results_1'
